In [1]:
import pandas as pd
import olspow as osp
import create_data as cd
import snowflake.connector

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = cd.gen_data(n_c=2000, mu_of_mu_c_cov_1=120, std_of_mu_c_cov_1=10, mu_of_c_std_cov_1=1,  std_of_c_std_cov_1=1)
s_mde = df[df['TREATED_CLUSTER'] == 1]['TREATMENT_EFFECT'].mean()*0.10
df.head(20)


,ID,CLUSTER_ID,NOBS_IN_CLUSTER,Y,TREATED_CLUSTER,TREATMENT_EFFECT,X1_VALUE,X1_COEF,X2_VALUE,X2_COEF,EPSILON
0,0,0,1,110.316338,0,0.000000,130.248248,0.608172,18.398494,1.647223,0.796587
1,1,1,1,112.891921,0,0.000000,117.181268,0.429031,38.379851,1.716424,-3.258591
2,2,2,1,92.651785,0,0.000000,122.178008,0.460963,14.944848,2.040693,5.834371
3,3,3,1,90.289311,1,9.237459,123.984634,0.467376,9.187816,2.384425,1.196744
4,4,4,3,90.225928,1,4.308410,112.554587,0.489163,25.951876,1.203370,-0.369778
5,5,4,3,83.231509,1,3.949461,112.693519,0.504078,24.265687,1.097701,-4.160769
6,6,4,3,96.551217,1,4.513362,113.003541,0.509841,23.086259,1.277060,4.941499
7,7,5,1,78.436331,1,3.107711,134.057248,0.425012,9.818488,1.535636,3.275103
8,8,6,1,104.065708,1,0.764079,133.660153,0.579588,29.119514,0.912462,-0.736605
9,9,7,1,88.350650,1,9.658740,112.252884,0.541425,20.501195,0.873039,0.017090


In [14]:
result = osp.solve_power(data=df,
                         endog='Y',
                        exog=['X1_VALUE'],
                        #  exog=['X1_VALUE', 'X2_VALUE'],
                         cluster='CLUSTER_ID',
                         ratio=0.1,
                         alpha=0.05,
                        #  mde=1,
                         power=0.8,
                         n=10000,
                         alternative="two-sided",
                         verbose=False)
result

The minimum detectable effect (MDE) was estimated as 2.049509473769423. 
Estimate was based on 2,350 observations across 2,000 clusters of historical data. 
The model specification used 1 covariates, assumes 2 coefficient estimates, and 2,000 degrees of freedom (alpha=0.025, beta=0.1).


2.049509473769423

In [ ]:
data = osp.solve_power(data=df, endog=response_var, exog=['X2_VALUE'],
                           cluster='CLUSTER_ID', ratio=psi, mde=s_mde, power=s_power, alpha=0.05, verbose=True)
data

In [9]:
"-"*80

'--------------------------------------------------------------------------------'

In [10]:
def _fetch_z_statistic(input, alternative):
    """
    Fetches the z-statistic based on the input and alternative.

    Parameters:
    input (float): The input value.
    alternative (str): The alternative hypothesis ("one-sided" or "two-sided").

    Returns:
    tuple: A tuple containing the z-statistic and the modified input value.
    """
    if alternative == "one-sided":
        return abs(st.norm.ppf(input)), input
    if alternative == "two-sided":
        return abs(st.norm.ppf(input / 2)), input / 2

In [12]:
import scipy.stats as st
input = 0.05
a, b = _fetch_z_statistic(input, "two-sided")

In [13]:
a

1.9599639845400545

In [14]:
b

0.025

In [ ]:
# import numpy as np
# import pandas as pd
# from scipy.stats import norm

# assignment_entity = 1
# covariate_1_values = []
# assignment_entity
# mu_of_mu_c_cov_1=120,  # the mean of the cluster means for covariate 1
# std_of_mu_c_cov_1=20,  # standard deviation of cluster means for covariate 1
# mu_of_c_std_cov_1=10,  # the mean of the cluster's standard deviation for covariate 1
# std_of_c_std_cov_1=5,  # the standard deviation of the cluster's standard deviation for covariate 1

# cluster_mean_cov_1_val = np.random.normal(
#     loc=mu_of_mu_c_cov_1, scale=std_of_mu_c_cov_1
# )
# cluster_std_cov_1_val = np.random.normal(
#     loc=mu_of_c_std_cov_1, scale=std_of_c_std_cov_1
# )
# covariate_1_values.append(
#     np.random.normal(
#         loc=cluster_mean_cov_1_val,
#         scale=cluster_std_cov_1_val,
#         size=max(assignment_entity, 1),
#     )
# )

In [ ]:
connection = snowflake.connector.connect(
    user="BENJAMINKNIGHT",
    account="INSTACART",
    authenticator="externalbrowser",
    role="instacart_developer_role",
)

In [ ]:
query_1 = """WITH base AS (
SELECT user_id,
       order_delivery_id,
       order_delivery_gmv_amt,
       initial_charge_amt_usd
FROM instadata.dwh.fact_order_delivery
WHERE delivery_created_date_time_utc BETWEEN '2023-12-20' AND '2024-01-20'
),

assignment_df AS (
SELECT
    DISTINCT user_id,
    CASE WHEN uniform(0::float, 1::float, random()) < 0.5
         THEN 1 ELSE 0 END AS treated
FROM base
),

child_df AS (
SELECT
    x.user_id,
    a.treated,
    x.order_delivery_id,
    x.order_delivery_gmv_amt,
    x.initial_charge_amt_usd,
    AVG(soias.availability_score) AS average_avail_score
FROM
    base AS x
JOIN assignment_df AS a
  ON a.user_id = x.user_id
JOIN instadata.dwh.fact_order_item AS foi
  ON foi.order_delivery_id = x.order_delivery_id
JOIN instadata.rds_data.shopper_order_item_availability_scores AS soias
  ON soias.order_item_id = foi.order_item_id
WHERE 1 = 1
 AND foi.order_date_pt BETWEEN '2023-12-18' AND '2024-01-22'
 AND soias.created_at BETWEEN '2023-12-18' AND '2024-01-22'
GROUP BY
    x.user_id,
    a.treated,
    x.order_delivery_id,
    x.order_delivery_gmv_amt,
    x.initial_charge_amt_usd
)

SELECT
    user_id,
    order_delivery_id,
    treated,
    CASE WHEN treated = 1 AND order_delivery_gmv_amt > 0.0
         THEN order_delivery_gmv_amt*1.01 ELSE order_delivery_gmv_amt
         END AS order_delivery_gmv_amt,
    initial_charge_amt_usd,
    average_avail_score
FROM child_df
ORDER BY uniform(0::float, 1::float, random())
LIMIT 100000"""

In [ ]:
query_2 = """WITH base AS (
SELECT user_id, order_delivery_id, order_delivery_gmv_amt, initial_charge_amt_usd
FROM instadata.dwh.fact_order_delivery
WHERE delivery_created_date_time_utc BETWEEN '2023-12-20' AND '2024-01-20'
),

assignment_df AS (
SELECT
    DISTINCT user_id,
    CASE WHEN uniform(0::float, 1::float, random()) < 0.5
         THEN 1 ELSE 0 END AS treated
FROM base
),

child_df AS (
SELECT
    x.user_id,
    a.treated,
    x.order_delivery_id,
    x.order_delivery_gmv_amt,
    x.initial_charge_amt_usd,
    AVG(soias.availability_score) AS average_avail_score
FROM
    base AS x
JOIN assignment_df AS a
  ON a.user_id = x.user_id
JOIN instadata.dwh.fact_order_item AS foi
  ON foi.order_delivery_id = x.order_delivery_id
JOIN instadata.rds_data.shopper_order_item_availability_scores AS soias
  ON soias.order_item_id = foi.order_item_id
WHERE 1 = 1
 AND foi.order_date_pt BETWEEN '2023-12-18' AND '2024-01-22'
 AND soias.created_at BETWEEN '2023-12-18' AND '2024-01-22'
GROUP BY
    x.user_id,
    a.treated,
    x.order_delivery_id,
    x.order_delivery_gmv_amt,
    x.initial_charge_amt_usd
)

SELECT
    user_id,
    treated,
    AVG(CASE WHEN treated = 1 AND order_delivery_gmv_amt > 0.0
         THEN order_delivery_gmv_amt*1.01 ELSE order_delivery_gmv_amt
         END) AS mean_gmv_per_order,
    AVG(initial_charge_amt_usd) AS avg_initial_charge_amt_usd,
    AVG(average_avail_score) AS average_avail_score
FROM child_df
GROUP BY 1,2
ORDER BY uniform(0::float, 1::float, random())
LIMIT 100000"""

In [ ]:
data_1 = pd.read_sql(query_1, connection)
data_1.head()

In [ ]:
data_2 = pd.read_sql(query_2, connection)
data_2.head()